# Referrence

https://youtu.be/kkQuywYNvO4?si=aXhLGANzrG3VStH5

# Acknowledge

เดี๋ยวมาเขียนอีกที มันเยอะ

# Library

In [ ]:
!pip install gdown
!pip install plotly
!pip install statsmodels

In [ ]:
import numpy as np
import pandas as pd

from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
    mean_absolute_percentage_error
)

import tensorflow as tf
import gdown
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go

# Load Data

In [ ]:
gdown.download(
    id="1-bNSbMCJvcxVsxvsiEHaPHIQBlLiFQgu",
    output="divorce_all_model.csv",
    quiet=False
)

In [ ]:
# โหลดข้อมูล (ทำ IQR มาแล้ว)
df = pd.read_csv("divorce_all_model.csv")

df = df.rename(columns={"y": "Divorce"})
df["ds"] = pd.to_datetime(df["ds"])
df = df.set_index('ds')
df = df.sort_index()

df

# Explore Data

In [ ]:
df.info()

In [ ]:
# ข้อมูลมีทั้งหมด 15508 บรรทัด 9 แถว
# ข้อมูลที่นำมาใช้ มีตั้งแต่ปี 2550 - 2567 (2007 - 2024)
# หย่าร้างกันต่ำสุดอยู่ที่ 0 คู่ แต่งงานต่ำสุดอยู่ที่ 0 คู่
# หย่าร้างกันสูงสุดอยู่ที่ 289 คู่ แต่งงานสูงสุดอยู่ที่ 1264 คู่
df.describe()

In [ ]:
fig = px.line(
    df.reset_index(), # <------------------------- ดึง Row มาดูเลย ตั้งแต่ปี 2021 - 2023
    x="ds",
    y="Divorce",
    title="Raw data"
)

fig.update_layout(
    template="plotly_white",
    hovermode="x unified"
)

fig.update_xaxes(
    dtick="M12",               # 👈 ปรับเป็น "M1" เพื่อให้โชว์ทุกเดือน หรือ "M3" สำหรับทุกไตรมาส
    # tickformat="%b %Y",       # 👈 %b คือชื่อเดือนย่อ (เช่น Jan, Feb) %Y คือปี
    tickformat="%Y",
    showgrid=True,
    tickangle=-45,            # 👈 เอียงเล็กน้อยเพื่อให้ชื่อเดือนไม่ซ้อนกัน
    hoverformat="%Y"    # 👈 Hover ให้เห็นละเอียดถึงระดับวัน/เดือน/ปี
)

fig.show()

# Function หา P-Value Observations Lags Used โดย ADF


In [ ]:
import pandas as pd
from statsmodels.tsa.stattools import adfuller, kpss

# =========================
# Augmented Dickey-Fuller
# =========================
def adf_test(timeseries):
    print("Results of Dickey-Fuller Test:")

    dftest = adfuller(timeseries, autolag="AIC")

    dfoutput = pd.Series(
        dftest[0:4],
        index=[
            "Test Statistic",
            "p-value",
            "# Lags Used",
            "Number of Observations Used"
        ]
    )

    for key, value in dftest[4].items():
        dfoutput[f"Critical Value ({key})"] = value

    print(dfoutput)
    print("-" * 50)




# Function หา P-Value และ Lags Used โดย KPSS

In [ ]:
# =========================
# KPSS Test
# =========================
def kpss_test(timeseries):
    print("Results of KPSS Test:")

    kpsstest = kpss(timeseries, regression="c", nlags="auto")

    kpss_output = pd.Series(
        kpsstest[0:3],
        index=[
            "Test Statistic",
            "p-value",
            "Lags Used"
        ]
    )

    for key, value in kpsstest[3].items():
        kpss_output[f"Critical Value ({key})"] = value

    print(kpss_output)
    print("-" * 50)


# หาค่า AIC ที่น้อยที่สุด



In [ ]:
df.info()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
plot_acf(df, lags=36)
plot_pacf(df, lags=36)
plt.show()

In [ ]:
import statsmodels.api as sm
decomposition = sm.tsa.seasonal_decompose(df, model='additive')
fig = decomposition.plot()
fig.set_size_inches(11,8)
adf_test(df)
kpss_test(df)

In [ ]:
# df_diff =df.diff(1).dif(2).dropna()
# plot_acf(df_diff, lags=30)
# plot_pacf(df_diff, lags=30)
# fig, ax = plt.subplots(figsize=(10,3))
# df_diff.plot(ax=ax)
# plt.show()
# adf_test(df_diff)
# kpss_test(df_diff)

# Grid Search

In [ ]:
# Set Parameter Optimizer
import itertools
p = [0,1,2]             # ค่า p สามารถปรับเปลี่ยนได้ แต่ก็จะใช้เวลามากขึ้นเหมือนกัน
d = [0,1,2]             # ค่า d สามารถปรับเปลี่ยนได้ แต่ก็จะใช้เวลามากขึ้นเหมือนกัน
q = [0,1,2]             # ค่า q สามารถปรับเปลี่ยนได้ แต่ก็จะใช้เวลามากขึ้นเหมือนกัน

# ดูว่าทั้งหมดมีกี่วิธี
params = list(itertools.product(p, d, q))
print(f"Number of models to test: {len(params)}")

In [ ]:
# หาค่า AIC แบบ Manual
from statsmodels.tsa.arima.model import ARIMA
import numpy as np

mod = ARIMA(df, order=(2,1,1))
results = mod.fit()
results.aic

In [ ]:
# Grid Search หา best parameter
from statsmodels.tsa.arima.model import ARIMA
import pandas as pd

results_list = []

for param in params:
    pdq = param[:3]
    try:
        model = ARIMA(df, order=pdq)
        res = model.fit()
        results_list.append({
            "p": pdq[0],
            "d": pdq[1],
            "q": pdq[2],
            "AIC": res.aic
        })
    except Exception as e:
        print(f"Failed ARIMA{pdq}: {e}")
        continue

df_results = pd.DataFrame(results_list)
df_results = df_results.sort_values("AIC").reset_index(drop=True)


In [ ]:
df_results.head(10)

In [ ]:
rank = 1
row = df_results.iloc[rank-1]

pdq = (row["p"], row["d"], row["q"])

print(f"Using ({pdq[0]},{pdq[1]},{pdq[2]})")

mod = ARIMA(df, order=pdq)
results = mod.fit()


# Data ควรเป็นยังไง
* Residual ควรมี mean ที่เป็น 0 และไม่มี Trend ที่ชัดเจน
* Histrogram ก็ควรมีค่าเป็น 0 ไม่เบ่ ซ้าย/ขวา
* Normal Q-Q (Linear) ควรเป็น ปกติเฉียง 45 ํ
* Correlation ควรมีความสัมพันธ์กับตัวเองเท่านั้น (อยู่ในกล่องทึบสีฟ้า)


In [ ]:
# Data หลังจากการทำ Grid Search เมื่อครู่
fig = results.plot_diagnostics(figsize=(10,6))
fig.tight_layout()

In [ ]:
pred = results.get_prediction(start=df.index[1], end=df.index[-1], dynamic=False)
comb = pd.concat([df, pred.predicted_mean], axis=1).dropna()
comb['error'] = comb.iloc[:,0] - comb.iloc[:,1]
comb['percentage' ] = comb['error']/comb.iloc[:,0]*100

MAE = comb['error'].abs().mean()
RMSE = np.sqrt((comb['error'] ** 2).mean())
MAPE = comb['percentage'].abs().mean()

print(f"Mean absolute error: {MAE:6.3f}")
print(f"Root mean squared error: {RMSE:6.3f}")
print(f"Mean absolute percentage error: {MAPE:6.3f}")

# หลังจากทำให้ Data นั้นเป็น Stationary มาที่สุดแล้วก็จึงจะสามารถเริ่ม Forecast ได้

In [ ]:
start_dt = df.index[10]
end_dt_data = df.index[-1]
print(df.index[10])
print(df.index[-1])

In [ ]:
num_forecast = 120  # <------------------------- จำนวนเดือนที่ใช้ทำนายอนาคต

freq = pd.infer_freq(df.index)

future_dates = pd.date_range(
    start=end_dt_data,
    periods=num_forecast + 1,
    freq=freq
)[1:]

end_dt = future_dates[-1]

print(future_dates)
print()
print(end_dt)

In [ ]:
plt.figsize=(12,8)
pred = results.get_prediction(start=pd.to_datetime(start_dt), end=pd.to_datetime(end_dt), dynamic=False)
pred_ci = pred.conf_int()
ax = df.plot(label='Observed')
pred.predicted_mean.plot(ax=ax, label='Prediction', alpha =.8, figsize=(14,6))
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha =.2)
ax.set_xlabel('Date')
ax.set_ylabel('Divorce')
ax.set_xlim(start_dt,end_dt)
yp_max = pred.predicted_mean.max()
yp_min = pred.predicted_mean.min()
yp_mean = pred.predicted_mean.mean()
ax.set_ylim(yp_min-0.1*yp_mean,yp_max+0.1*yp_mean)
plt.legend()
plt.show()

In [ ]:
import plotly.graph_objects as go

# 1. Prepare data (Assuming pred and pred_ci are already defined from your code)
mean = pred.predicted_mean
lower_ci = pred_ci.iloc[:, 0]
upper_ci = pred_ci.iloc[:, 1]

# 2. Calculate Y-axis limits (equivalent to your Matplotlib logic)
yp_max = mean.max()
yp_min = mean.min()
yp_mean = mean.mean()
y_limit_low = yp_min - 0.1 * yp_mean
y_limit_high = yp_max + 0.1 * yp_mean

# 3. Create the Figure
fig = go.Figure()

# --- Trace: Observed Data ---
fig.add_trace(go.Scatter(
    x=df.index, y=df['Divorce'],
    name='Observed',
    line=dict(color='blue')
))

# --- Trace: Prediction ---
fig.add_trace(go.Scatter(
    x=mean.index, y=mean,
    name='Prediction',
    line=dict(color='red', width=3)
))

# --- Trace: Confidence Interval (Upper Bound) ---
fig.add_trace(go.Scatter(
    x=upper_ci.index, y=upper_ci,
    line=dict(width=0),
    showlegend=False,
    hoverinfo='skip'
))

# --- Trace: Confidence Interval (Lower Bound + Fill) ---
fig.add_trace(go.Scatter(
    x=lower_ci.index, y=lower_ci,
    line=dict(width=0),
    fill='tonexty', # This fills the area between Upper and Lower
    fillcolor='rgba(0, 0, 0, 0.2)', # Gray with alpha
    name='Confidence Interval'
))

# 4. Update Layout (figsize, labels, and limits)
fig.update_layout(
    title="Divorce Prediction with Confidence Interval",
    xaxis_title="Date",
    yaxis_title="Divorce",
    width=1400,   # Equivalent to figsize 14
    height=600,   # Equivalent to figsize 6
    template="plotly_white",
    hovermode="x unified",
    xaxis_range=[start_dt, end_dt],
    yaxis_range=[y_limit_low, y_limit_high],
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01)
)

fig.show()

In [ ]:
fig = results.plot_diagnostics(figsize=(10, 6))
fig.tight_layout()

# SARIMA

In [ ]:
# โหลดข้อมูล (ทำ IQR มาแล้ว)
df = pd.read_csv("divorce_all_model.csv")

df = df.rename(columns={"y": "Divorce"})
df['ds'] = pd.to_datetime(df['ds'])
df = df.set_index('ds')
df = df.sort_index()

df

In [ ]:
max_sample = -1
df_avg = df['Divorce'].resample('MS').mean() # จริง ๆ ทั้ง Code นี้ไม่มีความจำเป็นเลย เพราะเรา Clean มาแล้ว
if max_sample > 0:
  df_avg = df_avg.iloc[-max_sample-1:-1]
# display(df_avg.describe())

#Convert result to DataFrame
df_avg =pd.DataFrame(df_avg)

In [ ]:
df_avg

In [ ]:
# df_diff =df.diff(1).dropna()
# .diff(12) คือค่า D ของ SARIMA ซึ่งถูกนับเป็น 1 Season
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
df_diff =df.diff(1).diff(24).dropna()

plot_acf(df_diff, lags=48)
plot_pacf(df_diff, lags=48)
fig, ax = plt.subplots(figsize=(10,3))
df_diff.plot(ax=ax)
plt.show()
adf_test(df_diff)
kpss_test(df_diff)

In [ ]:
df_diff

# SARIMA Grid Search

In [ ]:
import itertools

#               สามารถ Select ค่าเป็นแบบนี้ได้
#               แต่เวลาทำ Grid search หา Hyperparameter จะนานมาก ๆ
#               เพราะฉนั้นดูค่าจากด้านบนเอา แล้วประเมินค่าเอาดีกว่า

# ARIMA
p = [0,1,2,3,4,5]   # p = [0,1,2]
d = [0,1,2,3,4,5]   # d = [0,1,2]
q = [0,1,2,3,4,5]   # q = [0,1,2]

# SARIMA
P = [0,1,2,3,4,5]     # P = [0,1,2]
D = [0,1,2,3,4,5]     # D = [0,1,2]
Q = [0,1,2,3,4,5]     # Q = [0,1,2]

# จำนวน Lags
lag = [12]
params = list(itertools.product(p, d, q, P, D, Q, lag))
print(f"Number of models to test: {len(params)}")

In [ ]:
# เริ่มทำ Grid Search

'''
from statsmodels.tsa.statespace.sarimax import SARIMAX # <--------- S ARI MAX
import numpy as np
df_results = pd.DataFrame()
for param in params:
  pdq = param[0:3]
  PDQL = param[3:7]
  try:
    mod = SARIMAX(df, # <---------------------------------- เปลี่ยน Dataframe ตรงนี้
                  order=pdq, seasonal_order=PDQL)
    results = mod.fit(method = 'powell', # <------------------ Optimizer
                       start_params=np.random.random(7)) # <--- ถ้า Run ไม่ผ่าน มันก็เป็นเพราะตรงนี้ เพราะฉนั้นให้ Run ใหม่จะกว่าจะ Run ผ่าน
    data = {'param': pdq,
            'param_seasonal' : PDQL,
            'AIC' :results.aic }
    # DataFrame.append() ถูกถอดออกแล้ว (pandas ≥ 2.0)
    # df_results = df_results.append(data, ignore_index=True)

  except:
    continue
df_results = df_results.sort_values(by='AIC',ascending=True)
'''
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pandas as pd

results_list = []

for param in params:
    pdq  = param[0:3]
    PDQs = param[3:7]   # (P,D,Q,s)

    try:
        model = SARIMAX(
            df_diff['Divorce'],
            order=pdq,
            seasonal_order=PDQs,
            enforce_stationarity=False,
            enforce_invertibility=False
        )

        res = model.fit(disp=False)

        results_list.append({
            "p": pdq[0], "d": pdq[1], "q": pdq[2],
            "P": PDQs[0], "D": PDQs[1], "Q": PDQs[2], "s": PDQs[3],
            "AIC": res.aic
        })

    except Exception as e:
        print(f"❌ Failed SARIMA{pdq}x{PDQs}: {e}")
        continue

df_results = pd.DataFrame(results_list).sort_values("AIC").reset_index(drop=True)



In [ ]:
df_results

In [ ]:
'''
rank = 1
pdq = df_results.iloc[rank-1,1]
PDQL = df_results.iloc[rank-1,2]

print(f"Using ({pdq[0]},{pdq[1]},{pdq[2]})({PDQL[0]},{PDQL[1]},{PDQL[2]},{PDQL[3]})")

mod = SARIMAX(df, order=pdq, seasonal_order=PDQL)
results = mod. fit(method = 'powell',
                   start_params=np.random.random(7)) # <------------------------ ถ้า Run ไม่ผ่าน มันก็เป็นเพราะตรงนี้ เพราะฉนั้นให้ Run ใหม่จะกว่าจะ Run ผ่าน
'''
rank = 1
row = df_results.iloc[rank-1]

pdq  = (row["p"], row["d"], row["q"])
PDQL = (row["P"], row["D"], row["Q"], row["s"])

print(f"Using ({pdq[0]},{pdq[1]},{pdq[2]})"
      f"({PDQL[0]},{PDQL[1]},{PDQL[2]},{PDQL[3]})")

pdq  = tuple(df_results.iloc[rank-1, 0:3])
PDQL = tuple(df_results.iloc[rank-1, 3:7])

mod = SARIMAX(
    df['Divorce'],
    order=pdq,
    seasonal_order=PDQL,
    enforce_stationarity=False,
    enforce_invertibility=False
)

results = mod.fit(disp=False)


In [ ]:
fig = results.plot_diagnostics(figsize=(10,6))
fig.tight_layout()

In [ ]:
pred = results.get_prediction(start=df.index[1], end=df.index[-1], dynamic=False)
comb = pd.concat([df, pred.predicted_mean], axis=1).dropna()
comb['error'] = comb.iloc[:,0] - comb.iloc[:,1]
comb['percentage' ] = comb['error']/comb.iloc[:,0]*100

MAE = comb['error'].abs().mean()
RMSE = np.sqrt((comb['error'] ** 2).mean())
MAPE = comb['percentage' ].abs().mean()

print(f"Mean absolute error: {MAE:6.3f}")
print(f"Root mean squared error: {RMSE:6.3f}")
print(f"Mean absolute percentage error: {MAPE:6.3f}")

In [ ]:
num_forecast = 120 # <------------------------- ตั้งค่า Forecast ให้เท่ากับ 120 เดือน

df = df.sort_index()
freq = pd.infer_freq(df.index)

future_index = pd.date_range(
    start=df.index[-1],
    periods=num_forecast + 1,
    freq=freq
)[1:]

start_dt = df.index[10]
end_dt = future_index[-1]

print(start_dt)
print(end_dt)


In [ ]:
pred = results.get_prediction(start=pd.to_datetime(start_dt), end=pd.to_datetime(end_dt), dynamic=False)
pred_ci = pred.conf_int()
ax = df.plot(label='Observed')
pred.predicted_mean.plot(ax=ax, label='Prediction', alpha =.8, figsize=(14,6))
ax.fill_between(  pred_ci.index,
                  pred_ci.iloc[:, 0],
                  pred_ci.iloc[:, 1], color='k', alpha =.2)
ax.set_xlabel('Date')
ax.set_ylabel('Y')
ax.set_xlim(start_dt, end_dt)
yp_max = pred.predicted_mean.max()
yp_min = pred.predicted_mean.min()
yp_mean = pred.predicted_mean.mean()
ax.set_ylim(yp_min-0.1*yp_mean,yp_max+0.1*yp_mean)
plt.legend()
plt.show()

In [ ]:
pred.predicted_mean.min()*0.9

In [ ]:
display(pred.predicted_mean.loc[end_dt_data:end_dt])

# Rolling Forecast Evaluation

In [ ]:
import numpy as np
import pandas as pd

from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    mean_absolute_percentage_error
)

import matplotlib.pyplot as plt
import plotly.graph_objects as go

from statsmodels.tsa.statespace.sarimax import SARIMAX
import itertools


In [ ]:
df = pd.read_csv("divorce_all_model.csv")

df = df.rename(columns={"y": "Divorce"})
df["ds"] = pd.to_datetime(df["ds"])
df = df.set_index("ds")
df = df.sort_index()

y = df["Divorce"]

df.head()


In [ ]:
# df_diff =df.diff(1).dropna()
# .diff(12) คือค่า D ของ SARIMA ซึ่งถูกนับเป็น 1 Season
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
df_diff =df.diff(1).diff(24).dropna()

plot_acf(df_diff, lags=48)
plot_pacf(df_diff, lags=48)
fig, ax = plt.subplots(figsize=(10,3))
df_diff.plot(ax=ax)
plt.show()
adf_test(df_diff)
kpss_test(df_diff)

In [ ]:
#               สามารถ Select ค่าเป็นแบบนี้ได้
#               แต่เวลาทำ Grid search หา Hyperparameter จะนานมาก ๆ
#               เพราะฉนั้นดูค่าจากด้านบนเอา แล้วประเมินค่าเอาดีกว่า

# ARIMA
p = [0, 1, 2, 3]
d = [0, 1, 2, 3]
q = [0, 1, 2, 3]

# SARIMA
P = [0, 1, 2, 3]
D = [0, 1, 2, 3]
Q = [0, 1, 2, 3]

# Seasonal lag
lag = [12] # ปรับให้เป็น 12 เพราะต้องการให้มันมอง Time steps แค่ 12 เดือน
params = list(itertools.product(p, d, q, P, D, Q, lag))
print(f"Number of models to test: {len(params)}")


In [ ]:
# เริ่มทำ Grid Search

results_list = []

for param in params:
    pdq  = param[0:3]
    PDQs = param[3:7]   # (P,D,Q,s)

    try:
        model = SARIMAX(
            df['Divorce'],
            order=pdq,
            seasonal_order=PDQs,
            enforce_stationarity=False,
            enforce_invertibility=False
        )

        res = model.fit(disp=False)

        results_list.append({
            "p": pdq[0], "d": pdq[1], "q": pdq[2],
            "P": PDQs[0], "D": PDQs[1], "Q": PDQs[2], "s": PDQs[3],
            "AIC": res.aic
        })

    except Exception as e:
        print(f"❌ Failed SARIMA{pdq}x{PDQs}: {e}")
        continue

df_results = pd.DataFrame(results_list).sort_values("AIC").reset_index(drop=True)



In [ ]:
# เลือกโมเดลที่ดีที่สุด (AIC ต่ำสุด)
row = df_results.iloc[0]

pdq  = (row["p"], row["d"], row["q"])             #pdq = 1,1,2
PDQL = (row["P"], row["D"], row["Q"], row["s"])   #PDQL = 1,1,1,36

print("Using SARIMAX:", pdq, PDQL)

In [ ]:
pdq = (1, 1, 2)
PDQL = (1,1,1,12)

In [ ]:
def expanding_sarimax_full(
    y,
    pdq,
    PDQL,
    train_years=5,        # จำนวนปีที่เริ่ม Train
    test_years=1          # จำนวนปีที่ลอง Test
):
    summary_rows = []
    forecast_rows = []
    model_results = {}

    train_months = train_years * 12
    test_months  = test_years * 12

    train_start = y.index.min()
    train_end = train_start + pd.DateOffset(
        months=train_months
    ) - pd.DateOffset(days=1)

    round_no = 1

    while True:
        test_start = train_end + pd.DateOffset(days=1)
        test_end   = test_start + pd.DateOffset(
            months=test_months
        ) - pd.DateOffset(days=1)

        if test_end > y.index.max():
            break

        y_train = y.loc[train_start:train_end]
        y_test  = y.loc[test_start:test_end]

        if len(y_test) < test_months:
            break

        model = SARIMAX(
            y_train,
            order=pdq,
            seasonal_order=PDQL,
            enforce_stationarity=False,
            enforce_invertibility=False
        )

        fitted = model.fit(disp=False)

        forecast = fitted.forecast(steps=test_months)
        forecast.index = y_test.index

        mae  = mean_absolute_error(y_test, forecast)
        rmse = np.sqrt(mean_squared_error(y_test, forecast))
        mape = mean_absolute_percentage_error(y_test, forecast) * 100

        round_name = f"Round_{round_no}"

        summary_rows.append({
            "Round": round_name,
            "Train_Start": train_start,
            "Train_End": train_end,
            "Test_Start": test_start,
            "Test_End": test_end,
            "MAE": mae,
            "RMSE": rmse,
            "MAPE(%)": mape
        })

        model_results[round_name] = fitted

        for ds, yhat in forecast.items():
            forecast_rows.append({
                "Round": round_name,
                "Test_Year": ds.year,
                "ds": ds,
                "forecast": yhat
            })

        print(
            f'✔ {round_name} | '
            f'Train:"{train_start.date()}" - "{train_end.date()}" | '
            f'Test:"{test_start.date()}" - "{test_end.date()}" | '
            f'RMSE={rmse:.2f}'
        )

        train_end = train_end + pd.DateOffset(months=test_months)
        round_no += 1

    summary_df  = pd.DataFrame(summary_rows)
    forecast_df = pd.DataFrame(forecast_rows)

    return summary_df, forecast_df, model_results


In [ ]:
summary_df, forecast_df, model_results = expanding_sarimax_full(
    y=y,
    pdq=pdq,
    PDQL=PDQL,
    train_years=5,
    test_years=1
)


In [ ]:
summary_df


In [ ]:
summary_df[["MAE", "RMSE", "MAPE(%)"]].mean()


In [ ]:
summary_df.describe()


In [ ]:
fig = go.Figure()

# Actual
fig.add_trace(go.Scatter(
    x=df.index,
    y=df["Divorce"],
    mode="lines",
    name="Actual",
    line=dict(width=2)
))

# Forecast per Test Year
for year in sorted(forecast_df["Test_Year"].unique()):
    sub = forecast_df[forecast_df["Test_Year"] == year]

    fig.add_trace(go.Scatter(
        x=sub["ds"],
        y=sub["forecast"],
        mode="lines",
        name=f"Forecast {year}",
        line=dict(dash="dash")
    ))

fig.update_layout(
    title="Expanding Window SARIMAX Forecast",
    xaxis_title="Date",
    yaxis_title="Divorce",
    hovermode="x unified",
    height=600
)

fig.show()
